<p><p><font size=5 color="#5A5A5A" face = "Lano"><center>How did you become <b><font size=5 color="#FCAC45" face = "Lano">sick</font></b> ?</center></font><p>
<br>
<br>
<font size=3 color="#5A5A5A" face = "Lano"><B><u><font size=5 color="#FCAC45" face = "Lano">Summary</font></u></B></font><br><br>
<font size=3.2 color="#5A5A5A" face = "Lano">
  <b>I</b> - Data Introduction<br>
&nbsp;&nbsp;&nbsp;<b>A</b> - First taking of information<br>
&nbsp;&nbsp;&nbsp;<b>B</b> - Data Cleaning<br>
  <b>II</b> - Machine Learning Algorithms<br>
&nbsp;&nbsp;&nbsp;<b>A</b> - Random Forest<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>a</b>. Location & Date<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>b</b>. Food & Place<br>
&nbsp;&nbsp;&nbsp;<b>B</b> - Decision Tree<br>
</font>
<br>
<br>
<B>
<br>
<font size=4 color="#FCAC45" face = "Lano">
PART I: 
<font size=4 color ="#5A5A5A" face = "Lano">
DATA INTRODUCTION
</font>
</font>
</B>
<br>
<br>
<font size=4 color ="#5A5A5A" face = "Lano"><b><font size=4 color="#FCAC45" face = "Lano">I. A -</font> First Taking of Information</b></font>
<br>
<br>
<font color="#5A5A5A" size="3.2" face = "Lano">
We want to know what are the factors which cause an hospitalization after eating.<br> 
First we load our data and show the original length.<br>
Then we display the 5 first lines of our dataframe in order to know the content of our base 
</font>

In [ ]:
import pandas as pd
import numpy as np
from os import path

#Create first dataframe from poisoning.csv file
poisoning_df = pd.read_csv("data/poisoning.csv", sep=',')
print("poisoning.csv size :", len(poisoning_df))

poisoning_df.head()

<p><font size=4 color ="#5A5A5A" face = "Lano"><b><font size=4 color="#FCAC45" face = "Lano">I. B -</font> Data Cleaning</b></font></p>
<font color="#5A5A5A" size="3.2" face = "Lano">
We can now delete unused columns like years. We consider month could be important depending of epidemics<br>
Furthermore, we will delete "Fatalities" and "Illnesses" columns which are directly linked to our target column "Hospitalization" and which could falsified our prediction.
</font>

In [ ]:
del poisoning_df["Fatalities"] 
del poisoning_df["Illnesses"] 

<font color="#5A5A5A" size="3.2" face = "Lano">
Now we deleted unused columns, we can take a look at our target column which is naturally "Hospitalization".<br>
We will first check if our column contains NaN value.
</font>

In [ ]:
print("Hospitalizations NA:", pd.isnull(poisoning_df.Hospitalizations).values.any())

<font color="#5A5A5A" size="3.2" face = "Lano">
As target value cannot be NaN and as we can't replace NaN by mean or something else we will delete every Hospitalization lines containing NaN.<br>
We don't do dropna because we want to keep some information in other lines.<br>
Then we will globalize our target in two value, either their is Hospitalization: 1 or not: 0.<br>
We will display the new length of our dataframe.
</font>

In [ ]:
poisoning_df = poisoning_df[np.isfinite(poisoning_df['Hospitalizations'])]
poisoning_df['Hospitalizations_Global'] = np.where(poisoning_df.Hospitalizations > 0, 1, 0)
print("Hospitalization value count:")
print(poisoning_df.Hospitalizations_Global.value_counts())
print("\nNew size poisoning_df:", len(poisoning_df))

<font color="#5A5A5A" size="3.2" face = "Lano">
We will begin by work on columns which are easily workable. <br>
So we will create a 'Season' column according to the months in order to have less group for our Random Forest and knowing that in term of bacteria and virus, January and February are close.<br>
We don't forget to create a column encoded that we will use after for our Random Forest.
</font>

In [ ]:
def detect_season(df):
  season = df.Month
  if (season == "January") | (season == "February") | (season == "March"):
    return 0
  if (season == "April") | (season == "May") | (season == "June"):
    return 1
  if (season == "July") | (season == "August") | (season == "September"):
    return 2
  else:
    return 3

poisoning_df["Season"] = poisoning_df.apply(detect_season, axis=1)

<font color="#5A5A5A" size="3.2" face = "Lano">
Having do that we will do almost the same with States.<br>
The idea is to search on internet which are the states producing the most food, supposing that it import less and keep a good food quality.<br>
We will do exactly the same with fishing, repertoring states bordering the sea.
</font>

In [ ]:
def detect_producer(df):
  state = df.State
  #results found on http://beef2live.com/story-states-produce-food-value-0-107252
  if ((state == "California") | (state == "Iwoa") | (state == "Nebraska") 
      | (state == "Texas") | (state == "Minnesota")):
    return 1
  else:
    return 0

def detect_fishing(df):
  state = df.State
  #results found on https://en.wikipedia.org/wiki/Fishing_industry_in_the_United_States
  if ((state == "California") | (state == "Oregon") | (state == "Washington") 
      | (state == "Texas") | (state == "Louisiana") | (state == "Mississippi")| 
      (state == "North Carolina") | (state == "South Carolina") | 
      (state == "Georgia")| (state == "Virgina") | (state == "Maine") | 
      (state == "Maryland") | (state == "Michigan")| (state == "Wisconsin") | 
      (state == "Alaska")):
    return 1
  else:
    return 0

poisoning_df["ProducerState"] = poisoning_df.apply(detect_producer, axis=1)
poisoning_df["FishingState"] = poisoning_df.apply(detect_fishing, axis=1)

#Encode the state.
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
poisoning_df["StateEncoder"] = label_encoder.fit_transform(poisoning_df["State"])

<font color="#5A5A5A" size="3.2" face = "Lano">
Earlier we have seen that the "Location" column contains a lot of different values.<br>
The first thing we will do is to display the 50 most used values in this columns.
</font>

In [ ]:
LocValue = poisoning_df.Location.value_counts()
n = range(50)

for i in n: #Display 50 most used values in Location
  print(LocValue[[i]], "\n")

<font color="#5A5A5A" size="3.2" face = "Lano">
As we can see, there is a lot of different values in this column.<br>
For example, we can see that the most used location is "Restaurant" and we can see that this word is present in many other raws like "Fast Food Restaurant" or "Restaurant; Private Home/Residence".<br>
Our work will be to create a column "Restaurant" indicating if the restaurant word is present or not in the raw.<br>
<br>

First we will put every words in lower letters in order to avoid the casse.<br>
Secondly we will extract all 'restaurant' words in the 'Location' column. The "isRestaurant" column will have either NaN value or "restaurant" if Location contains 'restaurant'.<br>
Thirdly we just remplace NaN values by 'NO' in uppercase in order to have the same encoding for every columns.
</font>

In [ ]:
poisoning_df.Location = poisoning_df.Location.str.lower()
poisoning_df["isRestaurant"] = poisoning_df.Location.str.extract('.*(restaurant).*', expand=False).str.strip()
poisoning_df["isRestaurant"] = poisoning_df["isRestaurant"].fillna("NO")

<font color="#5A5A5A" size="3.2" face = "Lano">
Fourth we will encode our value 1 if it is a restaurant, otherwise (NO) 0.
</font>

In [ ]:
poisoning_df["isRestaurant"] = label_encoder.fit_transform(poisoning_df["isRestaurant"])
poisoning_df.head()

<font color="#5A5A5A" size="3.2" face = "Lano">
We created a magic function efficient for each column we will choose and for each word we will want.<br>
Having the column name "isXx" for the word "xx"<br>
We just have to create a list of columns containing the "isWord" we want to find.
</font>

In [ ]:
def create_columns(grepColumn, newColumns):
  grepWords = []
  for n in newColumns:
    n = n.lower()
    n = n[2:]
    grepWords.append(n)

  for n, t in zip(newColumns, grepWords):
    poisoning_df[n] = grepColumn.str.extract('.*(' + t +').*', expand=False).str.strip()

  for n in newColumns:
    poisoning_df[n] = poisoning_df[n].fillna("NON")

  for n in newColumns:
    poisoning_df[n] = label_encoder.fit_transform(poisoning_df[n])
#########END FUNCTION#########

newLocationColumn = ["isRestaurant","isResidence","isHome","isCatering","isFastFood","isBanquet",
                    "isSchool","isStore","isPrison","isNurse","isCamp","isReligious","isOffice",
                    "isFestival","isHospital", "isFarm"]

poisoning_df.Location = poisoning_df.Location.str.lower()
create_columns(poisoning_df.Location, newLocationColumn)

<font color="#5A5A5A" size="3.2" face = "Lano">
Now we can do exactly the same work with Food column.
</font>

In [ ]:
FoodValue = poisoning_df.Food.value_counts()
n = range(100)

for i in n: #Display 50 most used values in Food
  print(FoodValue[[i]], "\n")

<font color="#5A5A5A" size="3.2" face = "Lano">
We will create our column according to the most used values in food.<br>
And we can have an idea of what we did by displaying the 5 first rows of our dataframe. 
</font>

In [ ]:
newFoodColumn = ["isOysters", "isSalad", "isMultiple", "isUnspecified", "isSandwich", "isFish", 
                 "isPork", "isChicken", "isBbq", "isPotato", "isTuna", "isIce", "isDeli",
                "isPizza", "isMeat", "isMilk", "isCake", "isBeef", "isRice", "isEthnic", 
                 "isColeslaw", "isBeans", "isFruit", "isSushi", "isCheeseburger", "isEggs",
                "isSteak", "isCantaloupe", "isGuacamole", "isHam", "isTomato", "isLettuce", 
                 "isMushrooms"]   

poisoning_df.Food = poisoning_df.Food.str.lower()
create_columns(poisoning_df.Food, newFoodColumn)

poisoning_df.head()

<font color="#5A5A5A" size="3.2" face = "Lano">
Now, we have all the columns we need to begin our prediction. 
</font>
<br>
<br>
<B>
<br>
<font size=4 color="#FCAC45" face = "Lano">
PART II: 
<font size=4 color ="#5A5A5A" face = "Lano">
MACHINE LEARNING
</font>
</font>
</B>
<br>
<br>
<font size=4 color ="#5A5A5A" face = "Lano"><b><font size=4 color="#FCAC45" face = "Lano">II. A. a -</font> Random Forest: </b><em>Location & Date</em></font>
<br>
<br>
<font color="#5A5A5A" size="3.2" face = "Lano">
First, we will make a prediction using Random Forest algorithm on our main initial columns corresponding to location and date. <br>
And directly show the result voluntarily. In the next part our approach will be detailed.
</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier as RandomForest
from pandas import DataFrame
from sklearn.cross_validation import train_test_split

target = poisoning_df.Hospitalizations_Global

features_without_datacleaning = ["StateEncoder", "Season", "Year"]
predictors = poisoning_df[features_without_datacleaning]

split = train_test_split(poisoning_df, target, test_size=0.4, random_state=42)
data_train, data_test, target_train, target_test = split

model = RandomForest(n_estimators=10)
model.fit(data_train[features_without_datacleaning], target_train)

print(model.score(data_test[features_without_datacleaning], target_test))

<font size=4 color ="#5A5A5A" face = "Lano"><b><font size=4 color="#FCAC45" face = "Lano">II. A. b -</font> Random Forest: </b><em>Food & Place</em></font>
<br>
<br>
<font color="#5A5A5A" size="3.2" face = "Lano">
Now that we have done our prediction over initial data we will do the same with the columns we created for food and place. The idea is to make a prediction more complex over things a person can manage like the food he'll eat and the place he'll go.<br>
To do that we recovered in a same list both of location and food columns we created before, as feature.<br>
Using the same target we performed a bootstrap (recover 40% 15494->40%->6198 random data of poisoning_df data set) 
</font>

In [ ]:
features = []
for c, l in zip(newLocationColumn, newFoodColumn):
    features.append(c)
    features.append(l)

predictors = poisoning_df[features]

split = train_test_split(poisoning_df, target, test_size=0.4, random_state=42)
data_train, data_test, target_train, target_test = split
print('full shape:', poisoning_df.shape, '\ntest shape:', data_test.shape, '\ntrain shape:', data_train.shape)

<font color="#5A5A5A" size="3.2" face = "Lano">
Once our RandomForest will be performed, we will display the importance of each column.
</font>

In [ ]:
model = RandomForest(n_estimators=10)
model.fit(data_train[features], target_train)
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

for feature, imp, i in zip(features, importances, range(len(features))):
    print("  %d. %s -> %f" % (i+1, feature, imp))

<font color="#5A5A5A" size="3.2" face = "Lano">
We create now a dataframe in order to sort the most important feature by score, taking care to keep the error estimation for the plot you will see below.<br>
Then from this dataframe, we create another one keeping the 10 most important features.
</font>

In [ ]:
variable_name = []
variable_score = []
variable_std = []
for f, imp, mystd in zip(features, model.feature_importances_, std):
    variable_name.append(f)
    variable_score.append(imp)
    variable_std.append(mystd)
    
index = range(len(variable_name))
d = {
    'Variable_Name': variable_name, 
    'Variable_Score': variable_score,
    'Variable_Std': variable_std
    }
variable_importance_df = DataFrame(data=d, index=index)
variable_importance_df = variable_importance_df.sort_values(by = 'Variable_Score', ascending=False)
variable_importance_TOP10_df = variable_importance_df.head(10)
variable_importance_TOP10_df

<font color="#5A5A5A" size="3.2" face = "Lano">
From this 10 dataframe lines we create a plot showing well the importance of each variable one against the other.
</font>

In [ ]:
# Plot the feature importances of the RF
plt.figure()
plt.title("Feature importances")
plt.bar(
        range(len(variable_importance_TOP10_df)), 
        np.array(variable_importance_TOP10_df['Variable_Score']), 
        color="r", 
        yerr=np.array(variable_importance_TOP10_df['Variable_Std']), 
        align="center",
        width=0.5
       )
plt.xticks(
            range(len(variable_importance_TOP10_df)), 
            np.array(variable_importance_TOP10_df['Variable_Name']),
            fontsize=9
          )
plt.xlim([-1, len(variable_importance_TOP10_df)])
plt.show()

<font color="#5A5A5A" size="3.2" face = "Lano">
Now we can calculate our score prediction only according to the 10 most important features.<br>
We process to a new bootstrap in order to visualize the progression on a new random data train.<br>
The 'confusion matrix' is a square matrix whose diagonal (top left to bottom right) represents the number of values correctly predicted.<br>
In the same way, metrics.accuracy_score will calculate a prediction score.<br>
</font>

In [ ]:
import sklearn

features_TOP10 = np.array(variable_importance_TOP10_df['Variable_Name'])
predictors = poisoning_df[features_TOP10]
split = train_test_split(poisoning_df, target, test_size=0.4, random_state=42)
data_train, data_test, target_train, target_test = split

model = RandomForest(n_estimators=10)
model.fit(data_train[features_TOP10], target_train)
prediction = model.predict(data_test[features_TOP10])

print("Confusion matrix:\n", sklearn.metrics.confusion_matrix(target_test, prediction))
print("\nThe score:\n", model.score(data_test[features_TOP10], target_test))

<font color="#5A5A5A" size="3.2" face = "Lano">
From now our work will be to improve this pretty good score.<br> 
We will use get_features_importance function created in student notebook doing almost what we have done above and improve it returning an array composed of a dataframe and a score.
</font>

In [ ]:
def get_features_importance(arg_dataframe, arg_columntarget, arg_features, arg_topNumber):
  
  predictors = arg_dataframe[arg_features]#student_df[arg_features]
  target = arg_columntarget#student_df.Global_Marks

  split = train_test_split(arg_dataframe, target, test_size=0.4, random_state=42)
  data_train, data_test, target_train, target_test = split

  model = RandomForest(n_estimators=10)
  model.fit(data_train[arg_features], target_train)
  importances = model.feature_importances_
  std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
    
  variable_name = []
  variable_score = []
  variable_std = []
  for f, imp, mystd in zip(arg_features, model.feature_importances_, std):
    variable_name.append(f)
    variable_score.append(imp)
    variable_std.append(mystd)
    
  index = range(len(variable_name))
  d = {
    'Variable_Name': variable_name, 
    'Variable_Score': variable_score,
    'Variable_Std': variable_std
  }
  variable_importance_df = DataFrame(data=d, index=index)
  variable_importance_df = variable_importance_df.sort_values(by = 'Variable_Score', ascending=False)
  variable_importance_TOP_df = variable_importance_df.head(arg_topNumber)

  features_TOP = np.array(variable_importance_TOP_df['Variable_Name'])
  predictors = arg_dataframe[features_TOP]

  model = RandomForest(n_estimators=10)
  model.fit(data_train[features_TOP], target_train)
  prediction = model.predict(data_test[features_TOP])

  TOP_score = model.score(data_test[features_TOP], target_test)
    
  resultArray = [variable_importance_TOP_df, TOP_score]

  return(resultArray)
###END FUNCTION###

<font color="#5A5A5A" size="3.2" face = "Lano">
An idea I had in order to improve my score was to create 3 new columns.<br>
"Sea" composed of fish, oysters etc.<br>
A second "Vegetable" column and another one named "Meat".
</font>

In [ ]:
def sea_food(df):
  oysters = df.isOysters
  fish = df.isFish
  tuna = df.isTuna
  sushi = df.isSushi
  if (oysters == 1) | (fish == 1) | (tuna == 1) | (sushi == 1):
    return 1
  else:
    return 0
  
poisoning_df["Sea"] = poisoning_df.apply(sea_food, axis=1)

In [ ]:
def vegetable_food(df):
  salad = df.isSalad
  potato = df.isPotato
  beans = df.isBeans
  coleslaw = df.isColeslaw
  cantaloupe = df.isCantaloupe
  guacamole = df.isGuacamole
  tomato = df.isTomato
  lettuce = df.isLettuce
  mushrooms = df.isMushrooms
  if ((salad == 1) | (potato == 1) | (beans == 1) | (coleslaw == 1) 
      | (cantaloupe == 1) | (guacamole == 1) | (tomato == 1) | 
      (lettuce == 1)| (mushrooms == 1)):
    return 1
  else:
    return 0

poisoning_df["Vegetable"] = poisoning_df.apply(vegetable_food, axis=1)

In [ ]:
def meat_food(df):
  pork = df.isPork
  chicken = df.isChicken
  bbq = df.isBbq
  deli = df.isDeli
  meat = df.isMeat
  beef = df.isBeef
  cheeseburger = df.isCheeseburger
  ham = df.isHam
  if ((pork == 1) | (chicken == 1) | (bbq == 1) | (deli == 1) 
      | (meat == 1) | (beef == 1) | (cheeseburger == 1) | (ham == 1)):
    return 1
  else:
    return 0

poisoning_df["Meat"] = poisoning_df.apply(meat_food, axis=1)

<font color="#5A5A5A" size="3.2" face = "Lano">
Now we can add those 3 columns to our features.
</font>

In [ ]:
#Be careful to not run this cell two times.
newfeatures1 = []
newfeatures1 = features
newfeatures1.append("Sea")
newfeatures1.append("Vegetable")
newfeatures1.append("Meat")

<font color="#5A5A5A" size="3.2" face = "Lano">
And use our magic function taking new features as argument.
</font>

In [ ]:
RF_result = get_features_importance(poisoning_df, poisoning_df.Hospitalizations_Global, newfeatures1, 10)
print("Global score:", RF_result[1])
RF_result[0]

<font color="#5A5A5A" size="3.2" face = "Lano">
Another idea I had was to create a column highlighting the person who have eaten fish in a state which don't border the sea meaning that the fish was imported.
</font>

In [ ]:
def imported_fish(df):
  fish = df.isFish
  fishingstate = df.FishingState
  if (fish == 1) & (fishingstate == 0):
    return 1
  else:
    return 0

poisoning_df["Eat_Imported_Fish"] = poisoning_df.apply(imported_fish, axis=1)

In [ ]:
#Be careful to not run this cell two times.
newfeatures2 = []
newfeatures2 = newfeatures1
newfeatures2.append("Eat_Imported_Fish")

In [ ]:
RF_result = get_features_importance(poisoning_df, poisoning_df.Hospitalizations_Global, newfeatures2, 10)
print("Global score:", RF_result[1])
RF_result[0]

<font color="#5A5A5A" size="3.2" face = "Lano">
Unfortunately, this time my idea was not as good as I hoped because the new feature is not in the top 10.<br>
Nevertheless, the score is pretty good for data which was unworkable some cells ago.<br><br>
So with random forest we have succeeded a prediction improvement which finally reach a score of almost 75%, only using pandas skills and from only two "dirty" columns.
</font>

<font size=4 color ="#5A5A5A" face = "Lano"><b><font size=4 color="#FCAC45" face = "Lano">II. B -</font> Decision Tree</b></font>
<br>
<br>
<font color="#5A5A5A" size="3.2" face = "Lano">
There is a legend saying that it is very dangerous to eat oysters in a restaurant.<br>
So I decided to create a decision tree taking into acount those two features.
</font>

In [ ]:
# Need 'brew install graphviz' on terminal for MacOSX
# Or ‘sudo apt-get install graphviz’ for Linux
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from io import StringIO
from IPython.display import Image
import pydotplus

#make classifier with our main feature
main_feature = ["isRestaurant", "isOysters"]
features = main_feature
predictors = poisoning_df[features]
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, target, test_size=.4)
classifier = DecisionTreeClassifier(min_samples_split=20, random_state=10)
classifier = classifier.fit(pred_train, tar_train)

#Draw
print("Features:", main_feature)
print("Features: ['X[0]', 'X[1]']")
out = StringIO()
tree.export_graphviz(classifier, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
Image(graph.create_png())

<font color="#5A5A5A" size="3.2" face = "Lano">
Because of the bootstrap it's difficult to analyse the decision tree with you. <br>
But you can see that the lower "value" list represents those who ate oysters.<br>
Normally, in the last line, on the second box from the left there is those who ate oysters in at home and everywhere else than in a restaurant and on the last box those who ate oysters in a Restaurant.<br>
You can see that proportion show a bigger danger when you eat oysters at home meaning the legend is true...<br>
<br>
<font size=4 color="#FCAC45" face = "Lano"><b>•</b></font> This is all for this study. I hope you enjoyed it and you will continue to <font size=4 color="#FCAC45" face = "Lano"><b>eat</b></font> everything everywhere.
</font>